# Reanding Exif data from ./images

In [52]:
import os
from PIL import Image

dir = r"images/"
themes = os.listdir(dir)
data = []

for theme in themes: 
    if theme in {".DS_Store", "ExifData.json"}:
        pass   
    else:
        print(theme)
        for image in os.listdir(dir + theme):
            #print(image)
            if image.endswith(".jpg") :
                img_path =  dir + theme + '/'+ image
                # print(img_path)
                with Image.open(img_path) as img:
                    # print(img)
                    width, height = img.size
                    format = img.format
                    try:
                        orientation = img._getexif()[274]
                    except:
                        orientation = "no exif"
                    #print(f'{width=}x{height=} {format=} {orientation=}')
                    data.append({"theme": theme, "image": image, "width": width, "height": height, "format": format, "orientation": orientation})
                    
print(data[0:5])

Exoplanet
Pokemon
Moto
Voitures
[{'theme': 'Exoplanet', 'image': '55_Cancri_b.jpg', 'width': 1024, 'height': 768, 'format': 'PNG', 'orientation': 'no exif'}, {'theme': 'Exoplanet', 'image': 'Gliese_876_e.jpg', 'width': 1587, 'height': 919, 'format': 'PNG', 'orientation': 'no exif'}, {'theme': 'Exoplanet', 'image': 'XO-3b.jpg', 'width': 800, 'height': 500, 'format': 'JPEG', 'orientation': 'no exif'}, {'theme': 'Exoplanet', 'image': 'OGLE-2005-BLG-169Lb.jpg', 'width': 1271, 'height': 746, 'format': 'JPEG', 'orientation': 'no exif'}, {'theme': 'Exoplanet', 'image': 'Kepler-242b.jpg', 'width': 1980, 'height': 1200, 'format': 'JPEG', 'orientation': 'no exif'}]


## Get proiminant colors from images

In [53]:
from PIL import Image
import warnings
import numpy
import math
import matplotlib.pyplot as plot
from sklearn.cluster import MiniBatchKMeans

def getKmeans(img):
    # init
    img = Image.open(img) # open image
    imgfile = img.resize((int(img.size[0]/5), int(img.size[1]/5))) # resize
    numarray = numpy.array(imgfile.getdata(), numpy.uint8) # convert image to numpy array
    cluster_count = 5 # number of clusters
    clusters = MiniBatchKMeans(n_clusters=cluster_count, n_init="auto") # init kmeans
    clusters.fit(numarray) # fit kmeans
    npbins = numpy.arange(0, cluster_count + 1) # init bins
    histogram = numpy.histogram(clusters.labels_, bins=npbins) # get histogram
    #labels = numpy.unique(clusters.labels_) # get labels

    # getting colors
    color = []
    for i in range(cluster_count):
        color.append(
            "#%02x%02x%02x" % (math.ceil(clusters.cluster_centers_[i][0]),math.ceil(clusters.cluster_centers_[i][1]),math.ceil(clusters.cluster_centers_[i][2]),)
        )
    #barlist = plot.bar(labels, sorted(histogram[0], reverse=True), color=color)
    #plot.show()
    mapping = {"numbers": sorted(histogram[0], reverse=True), "colors": color}
    return (mapping)

getKmeans("./images/Exoplanet/2M1207_b.jpg")

{'numbers': [80952, 76247, 22307, 21412, 19282],
 'colors': ['#191c1f', '#a19081', '#7a6351', '#090a0c', '#403830']}

## Getting histogram from images

In [54]:
from PIL import Image
import matplotlib.pyplot as plot


def getDiagram(img):
    imgfile = Image.open("./images/Exoplanet/2M1207_b.jpg")
    histogram = imgfile.histogram()
    red = histogram[0:255]
    green = histogram[256:511]
    blue = histogram[512:767]
    # x = range(255)
    # y = []
    # for i in x:
    #     y.append((red[i], green[i], blue[i]))
    # figure,axes = plot.subplots()
    # axes.set_prop_cycle("color", ["red", "green", "blue"])
    # plot.plot(x, y)
    # plot.show()
    mapping = {"red": red, "green": green, "blue": blue}
    return mapping

getDiagram("./images/Exoplanet/2M1207_b.jpg")

{'red': [185917,
  64137,
  77972,
  98775,
  134976,
  133433,
  97798,
  88866,
  99499,
  86791,
  93429,
  125370,
  143397,
  119714,
  93507,
  118675,
  156700,
  171818,
  145261,
  121475,
  130494,
  155757,
  139709,
  122457,
  100407,
  85065,
  93007,
  103234,
  76943,
  67056,
  62337,
  50277,
  42024,
  33693,
  31057,
  30540,
  28750,
  31151,
  31665,
  30593,
  25363,
  24447,
  25664,
  25623,
  23405,
  21874,
  22350,
  22725,
  20634,
  18354,
  15473,
  14581,
  14282,
  14351,
  13147,
  12655,
  13639,
  14407,
  14483,
  15085,
  15519,
  14827,
  13800,
  13720,
  12240,
  11560,
  11530,
  9792,
  9362,
  9674,
  9477,
  8985,
  8755,
  8447,
  8787,
  7869,
  7560,
  7857,
  8521,
  8345,
  8487,
  9387,
  9833,
  9622,
  9852,
  9446,
  8727,
  8424,
  8355,
  8092,
  8132,
  8226,
  8038,
  7866,
  7654,
  6877,
  7245,
  7771,
  8431,
  8402,
  8403,
  7768,
  7147,
  6970,
  7555,
  8183,
  8414,
  8545,
  8108,
  8251,
  8537,
  8918,
  9471,
  100

## Adding diagram and Kmeans to exif data

In [55]:
for obj in data:
    print(obj["image"])
    obj["kmeans"] = getKmeans(dir + obj["theme"] + '/' + obj["image"])
    obj["diagram"] = getDiagram(dir + obj["theme"] + '/' + obj["image"])

55_Cancri_b.jpg
Gliese_876_e.jpg
XO-3b.jpg
OGLE-2005-BLG-169Lb.jpg
Kepler-242b.jpg
Gliese_581_e.jpg
HD_11977_b.jpg
HD_149026_b.jpg
Kepler-37b.jpg
KELT-9b.jpg
HAT-P-28b.jpg
WASP-39b.jpg
TRAPPIST-1h.jpg
Gliese_422_b.jpg
Gliese_876_d.jpg
Kepler-20f.jpg
GJ_3470_b.jpg
GJ_504_b.jpg
Gamma1_Leonis_b.jpg
PSR_B1620-26_b.jpg
Tau_Bootis_Ab.jpg
70_Virginis_b.jpg
Kepler-283_c.jpg
HD_20367_b.jpg
Epsilon_Eridani_b.jpg
OGLE-2016-BLG-1190Lb.jpg
Kepler-20e.jpg
55_Cancri_d.jpg
GJ_357_d.jpg
Gliese_876_c.jpg
Gliese_581_c.jpg
Beta_Pictoris_b.jpg
Proxima_Centauri_b.jpg
Gliese_581_b.jpg
HD_17156_b.jpg
Gliese_876_b.jpg
55_Cancri_e.jpg
51_Pegasi_b.jpg
Ross_128_b.jpg
Gliese_849_b.jpg
23_Librae_b.jpg
HD_189733_b.jpg
OGLE-2013-BLG-0341LB_b.jpg
Gliese_176_b.jpg
LkCa_15_b.jpg
Kepler-1647_b.jpg
XO-2b.jpg
Iota_Horologii_b.jpg
Gliese_357_b.jpg
55_Cancri_f.jpg
KOI-172.02.jpg
HD_40307_e.jpg
Gliese_86_b.jpg
Kepler-11b.jpg
Wolf_1061b.jpg
61_Virginis_b.jpg
CoRoT-9b.jpg
Gliese_3021_b.jpg
14_Herculis_c.jpg
HD_222582_b.jpg
K2-1

## Writing exif data into json file : ./images/ExifData.json

In [58]:
import json
from numpyencoder import NumpyEncoder

with open('./images/ExifData2.json', 'w') as f:
    json.dump(data, f, indent=4, cls=NumpyEncoder)